# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProjectAutoML'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AXNSBUE8U to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [2]:
import pandas as pd
df=pd.read_csv('train.csv',index_col=0)
print(df.head())
df.to_csv('trn.csv',index=False)
d=pd.read_csv('trn.csv')
d.head()

,1,2,3,4,5,6,7,8,9,y
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [5]:
# Import the data
from azureml.core.dataset import Dataset
import pandas as pd


key = 'TicTacToe data'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found")
    datastore = ws.get_default_datastore()
    #train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])

else:
    datastore = ws.get_default_datastore()
    datastore.upload_files(files = ['./trn.csv'])
    train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])
    train = train.register(ws,key)


Uploading an estimated of 1 files
Uploading ./trn.csv
Uploaded ./trn.csv, 1 files out of an estimated total of 1
Uploaded 1 files


# Define Compute instance

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

Creating a new compute cluster...
Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy",
                   "experiment_timeout_minutes":30,
                   "enable_early_stopping":True,
                   'n_cross_validations':4,
                   "max_concurrent_iterations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name ='y',**automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_217f0f70-4fe9-4b0e-8f7b-5282de6d783a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:    

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
bestRun, bestModel = remote_run.get_output()
model_name = bestRun.properties['model_name']

print('Best model: {}\n\n\nProperties of best model:{}'.format(model_name,bestModel))

ConfigException: ConfigException:
	Message: The model you attempted to retrieve requires 'xgboost' to be installed at '==0.90'. You have 'xgboost==1.3.3', please reinstall 'xgboost==0.90' (e.g. `pip install xgboost==0.90`) and rerun the previous command.
	InnerException: XGBoostError: [09:53:43] /mnt/xgboost/src/learner.cc:922: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.


Stack trace:
  [bt] (0) /anaconda/envs/azureml_py36/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x7c) [0x7f01b9f8664c]
  [bt] (1) /anaconda/envs/azure

In [11]:
#TODO: Save the best model

model_register= remote_run.register_model(model_name='bestmodel')

model_register.download(target_dir="outputs", exist_ok=True)

'outputs/model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
# Import Operational Libraries

from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
import requests
import json

In [13]:
config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                            memory_gb=4, 
                                            enable_app_insights=True,
                                            description='bestmodel')

model = Model(ws, 'bestmodel')


# Using the same environment for deployment
env_deploy = Environment.get(workspace=ws, name='AzureML-AutoML')

Inference_config = InferenceConfig(entry_script="score.py", environment=env_deploy)


# Deploy the best model
service = Model.deploy(workspace=ws, 
                       name='bestmodel', 
                       models=[model], 
                       inference_config=Inference_config, 
                       deployment_config=config)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [14]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"


In [15]:
# test the web service 

scoring_uri = service.scoring_uri

# Ans should be positive
data={'request':[
    {
        '1':'x',
        '2':'x',
        '3':'x',
        '4':'b',
        '5':'b',
        '6':'b',
        '7':'o',
        '8':'b',
        '9':'o',
    }
]}

# Convert to json string
data=json.dumps(data)
headers = {'Content-Type': 'application/json'}


# Make a request

responce = requests.post(scoring_uri, data, headers=headers)
print(responce.json())

MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

TODO: In the cell below, print the logs of the web service and delete the service

In [16]:

print(service.scoring_uri)
print(service.swagger_uri)

None
None
